In [1]:
import numpy as np
import math

In [2]:
def fast_exp(base, exponent, modulus):
  result = 1
  while exponent != 0:
    if exponent % 2 == 1:
      result = (result * base) % modulus
    base = base**2 % modulus
    exponent = exponent // 2
  return result

def extended_euclidean(m, n):
  if n == 0:
    return 1, 0, m
  x, y, g = extended_euclidean(n, m % n)
  return y, x - (m // n)*y, g

def inv_mod(a, modulus):
  s, _, g = extended_euclidean(a, modulus)
  assert g == 1, ValueError('the modular inverse does not exist')
  return s % modulus

def crt(remainders, moduli):
  assert len(remainders) == len(moduli), ValueError('the lists remainders and moduli are not the same length')
  assert len(remainders) > 0, ValueError('the list lengths must be greater than one')

  first_modulus = moduli[0]
  first_remainder = remainders[0]

  if len(remainders) == 1:
    return first_remainder % first_modulus, first_modulus
  
  consecutive_remainder, consecutive_modulus = crt(remainders[1:], moduli[1:])
  u, v, g = extended_euclidean(consecutive_modulus, first_modulus)

  assert g == 1, ValueError('the moduli are not relatively prime')

  return (first_remainder*consecutive_modulus*u + consecutive_remainder*first_modulus*v) % (first_modulus*consecutive_modulus), first_modulus*consecutive_modulus

def polynomial_root(exponent, a, modulus):
  u = inv_mod(exponent, modulus - 1)

  result = fast_exp(a, u, modulus)

  return result

def cf_float(x, n):
  cf = []
  for _ in range(n):
    cf.append(int(x))
    x = 1 / (x % 1)
  return cf

def continued_frac(num, den):
  cf = []
  while den != 0:
    cf.append(num//den)
    num, den = den, num % den
  return cf

def get_convergents(cf):
  convergents = [[0, 1], [1, 0]]
  for a in cf:
    hk2, hk1 = convergents[-2:]
    convergents.append([a*hk1[0] + hk2[0], a*hk1[1] + hk2[1]])
  return convergents[2:]

In [7]:
def crack_rsa(e, n):
  convergents = get_convergents(continued_frac(e, n))
  for convergent in convergents:
    k, d = convergent
    if k != 0 and (phi := (d * e - 1) // k) != 0:
      p_q = n - phi + 1
      disc = p_q**2 - 4*n
      if disc > 0:
        p, q = (p_q + math.isqrt(disc)) // 2, (p_q - math.isqrt(disc)) // 2
        if p*q == n:
          return d, p, q
  return -1, -1, -1

In [6]:
crack_rsa(107183767561739170366370713658887395868403028905518093258543, 189620700613125325959116839007395234454467716598457179234021)

(9576890767, 671998030559713968361666935769, 282174488599599500573849980909)